In [1]:
import pandas as pd
import requests
import time


In [ ]:

# Load your data
df = pd.read_csv('your_file.csv')

# NOAA API settings
NOAA_TOKEN = "YOUR_NOAA_API_TOKEN"



In [ ]:

BASE_URL = "https://www.ncei.noaa.gov/cdo-web/api/v2/data"
HEADERS = {"token": NOAA_TOKEN}

def fetch_weather_data(lat, lon, start_date, end_date, datasetid="GHCND", datatypeid="TAVG", limit=1000):
    # Find nearest station first (you could also cache this)
    stations_url = "https://www.ncei.noaa.gov/cdo-web/api/v2/stations"
    params = {
        "datasetid": datasetid,
        "limit": 1,
        "sortfield": "distance",
        "sortorder": "asc",
        "latitude": lat,
        "longitude": lon,
    }
    response = requests.get(stations_url, headers=HEADERS, params=params)
    response.raise_for_status()
    station_id = response.json()['results'][0]['id']
    
    # Now get the weather data
    data_params = {
        "datasetid": datasetid,
        "datatypeid": datatypeid,
        "stationid": station_id,
        "startdate": start_date,
        "enddate": end_date,
        "limit": limit,
        "units": "metric"
    }
    
    response = requests.get(BASE_URL, headers=HEADERS, params=data_params)
    if response.status_code != 200:
        return None
    return response.json().get("results", [])

# Example: process the DataFrame row by row and add results
weather_data = []

for _, row in df.iterrows():
    result = fetch_weather_data(
        lat=row["latitude"],
        lon=row["longitude"],
        start_date=row["start_date"],
        end_date=row["end_date"]
    )
    weather_data.append(result)
    time.sleep(1)  # Avoid rate limiting

df["weather"] = weather_data

# Save to new file
df.to_csv("weather_data_filled.csv", index=False)
